In [ ]:
!pip install openai

In [ ]:
import os
import openai
import pandas as pd

In [ ]:
import json

with open('/content/openai_cofig.json', 'r') as f:
    config = json.load(f)

In [ ]:
openai.api_key = config.get('api_key')

## Data Prep

In [ ]:
data = pd.read_excel('/content/Output_new_Q1.xlsx')
# data.head()

In [ ]:
data.columns

Index(['Business Name', 'Address', 'Question', 'Expected Response', 'Snippets',
       'Url', 'Machine result', 'Machine Snippet', 'Machine url', 'Validation',
       'Metrics', 'UW Review', 'UW Response', 'UW Comments', 'webtext',
       'New Context', 'Avg Sim', 'label', 'Score', 'Avg', 'Output'],
      dtype='object')

In [ ]:
data.columns

Index(['Business Name', 'Address', 'Question', 'Expected Response', 'Snippets',
       'Url', 'Machine result', 'Machine Snippet', 'Machine url', 'Validation',
       'Metrics', 'UW Review', 'UW Response', 'UW Comments', 'webtext',
       'New Context', 'Avg Sim', 'label', 'Score', 'Avg', 'Output'],
      dtype='object')

In [ ]:
print('Question:' , data['Question'][0])
data_subdf = data.loc[:,['Question','Business Name','Url','New Context','Expected Response','Avg Sim', 'Score', 'Avg']]
data_subdf['Metrics'] = data_subdf['Expected Response'].apply(lambda x: 'Yes' if x == 1 else 'No')
# data_subdf.head()

In [ ]:
for index,row in data_subdf.iterrows():
    text = row['New Context'].replace("  "," ")
    row['New Context'] = text

In [13]:
print(data_subdf.head())

                                            Question  ... Metrics
0  Does the applicant provide products or work re...  ...     Yes
1  Does the applicant provide products or work re...  ...     Yes
2  Does the applicant provide products or work re...  ...     Yes
3  Does the applicant provide products or work re...  ...     Yes
4  Does the applicant provide products or work re...  ...     Yes

[5 rows x 9 columns]


In [ ]:
#QUERY STRING
#question = "We provide products or work related to the operation, safety, navigation, or control of any of the following: aircraft, spacecraft, watercraft, rail transport, automobiles, or motorcycles?"
query_sentance = 'Meet Ground Control Systems.  5 SIMPLE STEPS FOR BOARD & SCOOTER SAFETY All without compromising safety for pedestrians or ease of use for riders.'


#PROMPT DESIGN
examples = [
            ['Vehicles are not anymore essentially mechanicals machines: Functional Safety is a fundamental requirement.  Function Safety and Security: Essential and Complementary Disciplines for Modern Systems  Functional Safety and Automotive Security','Yes'],
            ['All computing equipment and workstations present within the operational areas.  Operations are continuously monitored through supervisors, through software and through video cameras to prevent misuse. For instance, know which machines or equipment are most likely to need repairs and intervene before they break down','No'],
            ['in Core data, GPS Services, MapKit, Address Book, Push notification, audio/video streaming, etc.  We collect the following types of information: They include the following:','No'],
            ['Secure, protect and control your vessel from anywhere We can provide you the ability to monitor and control any equipment hardwired to your boat.. With our Smart Mates you can turn on/off motors, fans, pumps, de-humidifiers, heaters, lights, etc.  Remote Boat Monitoring & Control','Yes'],
            ['Rocket guidance & spacecraft navigation  Aerospace & Defense Manufacturing This should include the list of manufacturing steps and verification operations.','Yes'],
            ['Launch Lugs, Rail Buttons/Guides, Engine Hooks & Misc.  Engine Mount Kits, Engine Mount Tubes, Pusher Tubes, Engine Adapters, etc.  Model Rocket Recovery Devices (Parachutes, Streamers, etc.)','No']
           
           ]

#LABELS to CLASSIFY           
labels =  ['Yes','No']

classification_response = openai.Classification.create(
  search_model="ada",
  model="curie",
#   file = "file-bpNOGfmNKKUyqpGo0FDHohLx", #  If you use the file operation
  examples= examples,
  query=  query_sentance,
  labels=labels,
  #max_examples = 10, # Needed for filtering search as inputs
  return_prompt = True , # Used for Debugginf purpose,
#   return_metadata = True , # when are inputing custom dataset with FILE Opetation
#   expand=["completion"],
  logprobs=len(labels)+1, 
  #logit_bias = logit_bias  #we can boost the probabilities of predicting target labels by setting logit_bias explicitly {'1400': 100, '3363': 100} DICT of {token:bias weight}
)

In [ ]:


def get_classification_response(examples,labels,query_sentance):
    classification_response = openai.Classification.create(
    search_model="ada",
    model="curie",
    #   file = "file-bpNOGfmNKKUyqpGo0FDHohLx", #  If you use the file operation
    examples= examples,
    query=  query_sentance,
    labels=labels,
    #max_examples = 10, # Needed for filtering search as inputs
    return_prompt = True , # Used for Debugginf purpose,
    #   return_metadata = True , # when are inputing custom dataset with FILE Opetation
    #   expand=["completion"],
    logprobs=len(labels)+1, 
    #logit_bias = logit_bias  #we can boost the probabilities of predicting target labels by setting logit_bias explicitly {'1400': 100, '3363': 100} DICT of {token:bias weight}
    )

    return classification_response

In [ ]:
classification_response

<OpenAIObject classification at 0x7f93c5a2bb30> JSON: {
  "completion": {
    "choices": [
      {
        "finish_reason": "length",
        "index": 0,
        "logprobs": {
          "text_offset": [
            1735,
            1739
          ],
          "token_logprobs": [
            -0.51901585,
            -0.04376607
          ],
          "tokens": [
            " Yes",
            "\n"
          ],
          "top_logprobs": [
            {
              "\n": -6.707202,
              " No": -0.96542895,
              " Yes": -0.51901585
            },
            {
              "\n": -0.04376607,
              " ": -4.3531437,
              ",": -5.1899548
            }
          ]
        },
        "text": " Yes\n"
      }
    ],
    "created": 1639570690,
    "id": "cmpl-4FNWUiDnj2uXxcKoBjhMuXKb3y0sb",
    "model": "curie:2020-05-03",
    "object": "text_completion"
  },
  "label": "Yes",
  "model": "curie:2020-05-03",
  "object": "classification",
  "prompt": "Please 

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 8.7 MB/s 
     |████████████████████████████████| 596 kB 53.5 MB/s 
     |████████████████████████████████| 3.3 MB 16.1 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 61 kB 274 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import GPT2TokenizerFast

# Load the tokenizer.
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# Make sure the labels are formatted correctly.
labels =  ['Yes','No']
labels = [label.strip().lower().capitalize() for label in labels]

# Encode the labels with extra white space prepended.
labels_tokens = {label: tokenizer.encode(" " + label) for label in labels}
print(labels_tokens)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

{'Yes': [3363], 'No': [1400]}


In [ ]:
# #PROMPT DESIGN
# examples = [
#             ['Vehicles are not anymore essentially mechanicals machines: Functional Safety is a fundamental requirement.  Function Safety and Security: Essential and Complementary Disciplines for Modern Systems  Functional Safety and Automotive Security','Yes'],
#             ['All computing equipment and workstations present within the operational areas.  Operations are continuously monitored through supervisors, through software and through video cameras to prevent misuse. For instance, know which machines or equipment are most likely to need repairs and intervene before they break down','No'],
#             ['in Core data, GPS Services, MapKit, Address Book, Push notification, audio/video streaming, etc.  We collect the following types of information: They include the following:','No'],
#             ['Secure, protect and control your vessel from anywhere We can provide you the ability to monitor and control any equipment hardwired to your boat.. With our Smart Mates you can turn on/off motors, fans, pumps, de-humidifiers, heaters, lights, etc.  Remote Boat Monitoring & Control','Yes'],
#             ['Rocket guidance & spacecraft navigation  Aerospace & Defense Manufacturing This should include the list of manufacturing steps and verification operations.','Yes'],
#             [' Launch Lugs, Rail Buttons/Guides, Engine Hooks & Misc.  Engine Mount Kits, Engine Mount Tubes, Pusher Tubes, Engine Adapters, etc.  Model Rocket Recovery Devices (Parachutes, Streamers, etc.)','No']
           
#            ]

# #LABELS to CLASSIFY           
# labels =  ['Yes','No']

# classification_response = openai.Classification.create(
#   search_model="ada",
#   model="curie",
# #   file = "file-bpNOGfmNKKUyqpGo0FDHohLx", #  If you use the file operation
#   examples= examples,
#   query=  query_sentance,
#   labels=labels,
#   #max_examples = 10, # Needed for filtering search as inputs
#   return_prompt = True , # Used for Debugginf purpose,
# #   return_metadata = True , # when are inputing custom dataset with FILE Opetation
# #   expand=["completion"],
#   logprobs=len(labels)+1, 
# )

In [ ]:
import numpy as np

# Take the starting tokens for probability estimation.
# Labels should have distinct starting tokens.
# Here tokens are case-sensitive.
first_token_to_label = {tokens[0]: label for label, tokens in labels_tokens.items()}

#taking top log_probs from the API response
top_logprobs = classification_response["completion"]["choices"][0]["logprobs"]["top_logprobs"][0]

# Creating a Dict of Token and the Log_Probs with np.exp()
token_probs = {
    tokenizer.encode(token)[0]: np.exp(logp) 
    for token, logp in top_logprobs.items()
}

# Mapping the tokens with the Labels
label_probs = {
    first_token_to_label[token]: prob 
    for token, prob in token_probs.items()
    if token in first_token_to_label
}

# Fill in the probability for the special "Unknown" label. i.e (label+1) log_prob
if sum(label_probs.values()) < 1.0:
    label_probs["Unknown"] = 1.0 - sum(label_probs.values())

print(label_probs)


{'No': 0.38081981204089094, 'Yes': 0.5951059333737537, 'Unknown': 0.024074254585355392}


In [ ]:
# label_probs

{'No': 0.38081981204089094,
 'Unknown': 0.024074254585355392,
 'Yes': 0.5951059333737537}

#Putting it Together!

In [15]:
data_subdf.columns

Index(['Question', 'Business Name', 'Url', 'New Context', 'Expected Response',
       'Avg Sim', 'Score', 'Avg', 'Metrics'],
      dtype='object')

In [16]:
X = data_subdf.iloc[:,:-1]
Y = data_subdf.iloc[:,-1]

#### Creating Test Train Split

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=42)

In [23]:
train_feed = pd.DataFrame({'text':X_train['New Context'],'Metric':y_train}).reset_index(drop=True)

In [56]:
train_feed = pd.concat([X_train,pd.DataFrame({'Metric':y_train})], axis=1).reset_index(drop=True)

In [60]:
print(train_feed.columns)

Index(['Question', 'Business Name', 'Url', 'New Context', 'Expected Response',
       'Avg Sim', 'Score', 'Avg', 'Metric'],
      dtype='object')


In [26]:
example_feed = pd.DataFrame({'text':X_test['New Context'],'Metric':y_test}).reset_index(drop=True)

In [27]:
print(example_feed)

                                                text Metric
0  So many businesses taken part and exhibited th...     No
1   We strive to bring you the best in Radio Cont...     No
2            navigation, command and control, and...    Yes
3  Our system environments are the following:  We...     No
4   Vehicle status, control and GPS navigation  M...    Yes
5   To cope with the technological tide, we must ...     No
6   Secure, protect and control your vessel from ...    Yes


In [39]:
import numpy as np
from transformers import GPT2TokenizerFast


query_sentance = 'Meet Ground Control Systems.  5 SIMPLE STEPS FOR BOARD & SCOOTER SAFETY All without compromising safety for pedestrians or ease of use for riders.'


#PROMPT DESIGN
# examples = [
#             ['Vehicles are not anymore essentially mechanicals machines: Functional Safety is a fundamental requirement.  Function Safety and Security: Essential and Complementary Disciplines for Modern Systems  Functional Safety and Automotive Security','Yes'],
#             ['All computing equipment and workstations present within the operational areas.  Operations are continuously monitored through supervisors, through software and through video cameras to prevent misuse. For instance, know which machines or equipment are most likely to need repairs and intervene before they break down','No'],
#             ['in Core data, GPS Services, MapKit, Address Book, Push notification, audio/video streaming, etc.  We collect the following types of information: They include the following:','No'],
#             ['Secure, protect and control your vessel from anywhere We can provide you the ability to monitor and control any equipment hardwired to your boat.. With our Smart Mates you can turn on/off motors, fans, pumps, de-humidifiers, heaters, lights, etc.  Remote Boat Monitoring & Control','Yes'],
#             ['Rocket guidance & spacecraft navigation  Aerospace & Defense Manufacturing This should include the list of manufacturing steps and verification operations.','Yes'],
#             [' Launch Lugs, Rail Buttons/Guides, Engine Hooks & Misc.  Engine Mount Kits, Engine Mount Tubes, Pusher Tubes, Engine Adapters, etc.  Model Rocket Recovery Devices (Parachutes, Streamers, etc.)','No']
           
#            ]

examples = example_feed.values.tolist()

#LABELS to CLASSIFY           
labels =  ['Yes','No']



def get_classification_response(examples,labels,query_sentance):
    classification_response = openai.Classification.create(
    search_model="ada", # Search Model to be used  
    model="curie", 
    #   file = "file-bpNOGfmNKKUyqpGo0FDHohLx", #  If you use the file operation
    examples= examples,
    query=  query_sentance,
    labels=labels,
    #max_examples = 10, # Needed for filtering search as inputs
    return_prompt = True , # Used for Debugginf purpose,
    #   return_metadata = True , # when are inputing custom dataset with FILE Opetation
    #   expand=["completion"],
    logprobs=len(labels)+1, 
    #logit_bias = logit_bias  #we can boost the probabilities of predicting target labels by setting logit_bias explicitly {'1400': 100, '3363': 100} DICT of {token:bias weight}
    )

    return classification_response

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def get_label_probabilities(labels,classification_response,tokenizer):
    # Load the tokenizer.
    

    # Make sure the labels are formatted correctly.
    labels =  ['Yes','No']
    labels = [label.strip().lower().capitalize() for label in labels]

    # Encode the labels with extra white space prepended.
    labels_tokens = {label: tokenizer.encode(" " + label) for label in labels}
    # print(labels_tokens)

    first_token_to_label = {tokens[0]: label for label, tokens in labels_tokens.items()}

    #taking top log_probs from the API response
    top_logprobs = classification_response["completion"]["choices"][0]["logprobs"]["top_logprobs"][0]

    # Creating a Dict of Token and the Log_Probs with np.exp()
    token_probs = {
        tokenizer.encode(token)[0]: np.exp(logp) 
        for token, logp in top_logprobs.items()
    }

    # Mapping the tokens with the Labels
    label_probs = {
        first_token_to_label[token]: prob 
        for token, prob in token_probs.items()
        if token in first_token_to_label
    }

    # Fill in the probability for the special "Unknown" label. i.e (label+1) log_prob
    if sum(label_probs.values()) < 1.0:
        label_probs["Unknown"] = 1.0 - sum(label_probs.values())

    return label_probs


In [ ]:
# classification_response = get_classification_response(examples,labels,query_sentance)

In [ ]:
# label_probs = get_label_probabilities(labels,classification_response)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# label_probs

{'No': 0.3823460310333291,
 'Unknown': 0.023959527334891595,
 'Yes': 0.5936944416317793}

In [ ]:
# ordered_label_porbs = sorted(label_probs.items(), key=lambda item: item[1], reverse = True)

In [ ]:
# ordered_label_porbs[0]

('Yes', 0.5936944416317793)

In [29]:
print(examples)

[['So many businesses taken part and exhibited the latest technologies like AR/VR, Artificial Intelligence, Smart Living, Wearable, Robotics and many more. range of industries and businesses. website development, AR-VR Development, Game Development, Blockchain Development and much', 'No'], [' We strive to bring you the best in Radio Control Sailing!  Long Beach RC was started to help support and grow Radio Control Sailing. Boat Kits | Long Beach RC', 'No'], ['          navigation, command and control, and communications systems.           aircraft, artificial satellites, and spacecraft.           aircrafts, marine and handheld.', 'Yes'], ['Our system environments are the following:  We will begin with planning, taking into consideration your desired plan and vision. Here are a few examples of how we accomplish that:', 'No'], [' Vehicle status, control and GPS navigation  Military Air, Land and Sea  GPS navigation, mapping and surveillance', 'Yes'], [' To cope with the technological tid

In [35]:
data_subdf.columns

Index(['Question', 'Business Name', 'Url', 'New Context', 'Expected Response',
       'Avg Sim', 'Score', 'Avg', 'Metrics'],
      dtype='object')

In [62]:
from urllib.request import HTTPError

final_output = []

final_output_df = train_feed.copy()

# final_output_df = final_output_df[:4]

for index,row in final_output_df.iterrows():
    pred_output = {}
    query_sentance = row['New Context'].replace("  "," ").lstrip().rstrip()
    print(query_sentance)
    classification_response = get_classification_response(examples,labels,query_sentance)
   
    label_probs = get_label_probabilities(labels,classification_response,tokenizer)
    print(label_probs)
    pred_output['label_scores'] = label_probs
    ordered_label_porbs = sorted(label_probs.items(), key=lambda item: item[1], reverse = True)
    pred_output['pred_label'] = ordered_label_porbs[0][0]
    pred_output['pred_label_Score'] = ordered_label_porbs[0][1]


    # df.loc[index, 'd']
    final_output.append(pred_output)




Ground and sea based navigation aids, distance measuring and direction finding equipment. WORLD LEADER IN FLIGHT CONTROL SYSTEMS AND CRITICAL CONTROL PRODUCTS Navigation Aids and Surveillance Systems
{'No': 0.4004567132851452, 'Yes': 0.5861924384822425, 'Unknown': 0.013350848232612345}
Product Development & Management Solutions environment will display more detailed information about the error that occurred. Product development as a service
{'No': 0.5602944499259147, 'Yes': 0.42883043268379284, 'Unknown': 0.010875117390292455}
The system reads various types of vehicle information and controls the motor depending on the vehicle status in order to improve the driving performance. The system manages start control for high voltage batteries and DC/DC converters, charger communication, vehicle sensor input, motor output and various load outputs. Vehicles which operate using electricity as their primary energy source and which can accelerate smoothly through use of an electric motor.
{'No': 

In [63]:
final_output_df.columns

Index(['Question', 'Business Name', 'Url', 'New Context', 'Expected Response',
       'Avg Sim', 'Score', 'Avg', 'Metric'],
      dtype='object')

In [47]:
output_df = pd.concat([final_output_df,pd.DataFrame(final_output)], axis=1)

In [49]:
print(output_df)

                                             Question  ... pred_label_Score
0   Does the applicant provide products or work re...  ...         0.754524
1   Does the applicant provide products or work re...  ...         0.513109
2   Does the applicant provide products or work re...  ...         0.605675
3   Does the applicant provide products or work re...  ...         0.554658
4   Does the applicant provide products or work re...  ...         0.577209
..                                                ...  ...              ...
61  Does the applicant provide products or work re...  ...         0.517717
62  Does the applicant provide products or work re...  ...         0.885335
63  Does the applicant provide products or work re...  ...         0.709027
64  Does the applicant provide products or work re...  ...         0.606273
65  Does the applicant provide products or work re...  ...         0.560597

[66 rows x 12 columns]


In [64]:
output_df.to_excel('GPT3_Batch2_Q1_trainset_v2.xlsx',index=False)
print(output_df)

                                             Question  ... pred_label_Score
0   Does the applicant provide products or work re...  ...         0.754524
1   Does the applicant provide products or work re...  ...         0.513109
2   Does the applicant provide products or work re...  ...         0.605675
3   Does the applicant provide products or work re...  ...         0.554658
4   Does the applicant provide products or work re...  ...         0.577209
..                                                ...  ...              ...
61  Does the applicant provide products or work re...  ...         0.517717
62  Does the applicant provide products or work re...  ...         0.885335
63  Does the applicant provide products or work re...  ...         0.709027
64  Does the applicant provide products or work re...  ...         0.606273
65  Does the applicant provide products or work re...  ...         0.560597

[66 rows x 12 columns]


# Accuracy Scores

In [65]:
from sklearn.metrics import precision_score,recall_score,accuracy_score


prec_score = precision_score(output_df['Metrics'],output_df['pred_label'] , pos_label='Yes')

print('Precision_Score',prec_score)


Precision_Score 0.84375


In [66]:
acc_score = accuracy_score(output_df['Metrics'],output_df['pred_label'] )

print('accuracy_score',acc_score)


accuracy_score 0.803030303030303


In [67]:
recall_score = recall_score(output_df['Metrics'],output_df['pred_label'] ,pos_label='Yes')

print('recall_score',recall_score)

recall_score 0.7714285714285715
